
# 02 — Pipeline de Docking Molecular con Estructuras Representativas

**Propósito:** Tomar las estructuras representativas extraídas del cuaderno 01 y ejecutar un pipeline de preparación y docking contra la interfase de la proteína objetivo.  

**Contenido:** preparación del receptor, preparación de ligandos (centroides), definición de la caja, ejecución del docking (AutoDock Vina / Smina), post-procesado y exportación de resultados.



## Instalación 

Ejecutar en terminal  para crear el entorno recomendado:


conda create -n docking python=3.10 -c conda-forge -y
conda activate md-docking
mamba install -c conda-forge mdtraj mdanalysis openbabel rdkit nglview py3dmol vina biopython -y
python -m pip install pbxplore weblogo
python -m ipykernel install --user --name md-docking --display-name "Python (md-docking)"
```



## Índice

1. Configuración y lectura de rutas  
2. Preparación del receptor (limpieza, protonación, definición de interfase)  
3. Preparación de ligandos (centroides): conversión a PDBQT / minimización rápida  
4. Definición de la caja de docking (centro y dimensiones)  
5. Ejecución del docking (Vina / Smina) — ejemplo por ligando  
6. Post-procesado: scores, RMSD entre poses, contactos con la interfase  
7. Visualización y exportación de resultados  
8. Conclusiones y próximos pasos


## 1. Configuración y lectura de rutas

In [ ]:
import os
from pathlib import Path
import json

# Rutas 
PROJECT_ROOT = Path.cwd()
CENTROIDS_DIR = PROJECT_ROOT / "results" / "md_analysis" / "centroids"
RECEPTOR_PDB = PROJECT_ROOT / "data" / "receptor" / "receptor_clean.pdb"  # receptor (limpio)
OUTPUT_DIR = PROJECT_ROOT / "results" / "docking"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Parámetros de docking 
DOCKING_ENGINE = "vina"   # 'vina' o 'smina' (si instalas smina)
EXHAUSTIVENESS = 8
NUM_POSES = 10

print("Centroids dir:", CENTROIDS_DIR)
print("Receptor:", RECEPTOR_PDB)
print("Output:", OUTPUT_DIR)


## 2. Preparación del receptor

In [ ]:

# Ejemplo de limpieza simple con Biopython/MDAnalysis (adáptalo)
# NOTA: estas líneas son para ejecutar dentro del notebook cuando tengas las librerías instaladas.

import MDAnalysis as mda
from MDAnalysis.coordinates import PDB

# Cargar receptor (ejemplo)
u = mda.Universe(str(RECEPTOR_PDB))
print(u.atoms.n_atoms, "átomos en la estructura del receptor")

# Ejemplo: eliminar aguas y ligandos (define tu selección)
protein = u.select_atoms("protein")
# escribir PDB limpio
clean_receptor = OUTPUT_DIR / "receptor_clean_for_docking.pdb"
with mda.Writer(str(clean_receptor), protein.n_atoms) as W:
    W.write(protein)

print("Receptor limpio guardado en:", clean_receptor)



## 3. Preparación de ligandos (centroides)

In [ ]:
# Convertir centroides PDB -> PDBQT usando OpenBabel (ejemplo)
# Asegúrate de tener openbabel/obabel instalado en tu entorno.

import subprocess
centroid_files = sorted([p for p in (CENTROIDS_DIR).glob("*.pdb")])

prepared_ligands_dir = OUTPUT_DIR / "ligands_prepared"
prepared_ligands_dir.mkdir(parents=True, exist_ok=True)

for pdb in centroid_files:
    base = pdb.stem
    pdbqt_out = prepared_ligands_dir / f"{base}.pdbqt"
    # ejemplo comando obabel (ajusta si usas obabel o obconformer)
    cmd = f"obabel -ipdb {pdb} -opdbqt -O {pdbqt_out} --addhydrogens"
    print("Running:", cmd)
    subprocess.run(cmd, shell=True, check=False)
print("Ligandos convertidos (PDBQT) en:", prepared_ligands_dir)


## 4. Definición de la caja de docking

In [ ]:

# Definir caja: dos opciones comunes
#  - Centro en coordenadas promedio de residuos de la interfase
#  - Centro en atomos específicos (p. ej. CA de residuo X)

import numpy as np
import MDAnalysis as mda

u = mda.Universe(str(RECEPTOR_PDB))

# Ajusta la selección de interfase según tu receptor
interface_sel = u.select_atoms("resid 100:110 and name CA")  # <- ajustar
coords = interface_sel.positions
center = coords.mean(axis=0)
print("Centro de la caja (x,y,z):", center)

# Dimensiones de la caja (Å) - ajustar según tamaño del ligando/interfase
size_x = 20.0
size_y = 20.0
size_z = 20.0

# Guardar parámetros
box_params = {"center": center.tolist(), "size": [size_x, size_y, size_z]}
with open(OUTPUT_DIR / "box_params.json", "w") as f:
    json.dump(box_params, f, indent=2)
print("Box params saved:", OUTPUT_DIR / "box_params.json")


## 5. Ejecución del docking

In [ ]:
# Ejecutar AutoDock Vina para cada ligando (ejemplo)

import subprocess
from pathlib import Path

prepared_ligands = sorted((OUTPUT_DIR / "ligands_prepared").glob("*.pdbqt"))
receptor_pdbqt = OUTPUT_DIR / "receptor_clean_for_docking.pdbqt"

# Convertir receptor a PDBQT (si usas AutoDockTools/babel)
# Aquí asumimos que ya tienes receptor_pdbqt; de lo contrario, conviértelo con obabel o AutoDockTools

docking_results_dir = OUTPUT_DIR / "poses"
docking_results_dir.mkdir(parents=True, exist_ok=True)

# Cargar box params
with open(OUTPUT_DIR / "box_params.json") as f:
    box_params = json.load(f)

center = box_params["center"]
size = box_params["size"]

for lig in prepared_ligands:
    out_prefix = docking_results_dir / lig.stem
    out_pdbqt = out_prefix.with_suffix(".out.pdbqt")
    log_txt = out_prefix.with_suffix(".log.txt")
    cmd = f"vina --receptor {receptor_pdbqt} --ligand {lig} --center_x {center[0]} --center_y {center[1]} --center_z {center[2]} --size_x {size[0]} --size_y {size[1]} --size_z {size[2]} --exhaustiveness {EXHAUSTIVENESS} --num_modes {NUM_POSES} --out {out_pdbqt} --log {log_txt}"
    print("Running:", cmd)
    subprocess.run(cmd, shell=True, check=False)
print("Docking finished. Results in:", docking_results_dir)


## 6. Post-procesado y análisis de resultados

In [ ]:
# Leer logs / extraer scores y generar tabla resumen

import re
import pandas as pd
from pathlib import Path

docking_dir = OUTPUT_DIR / "poses"
summary = []

for out in sorted(docking_dir.glob("*.log.txt")):
    ligand_name = out.stem
    score = None
    with open(out) as f:
        for line in f:
            # Vina log contains lines like: "1  -7.5  0.000  0.000"
            m = re.match(r'\s*1\s+([\-0-9\.]+)\s+', line)
            if m:
                score = float(m.group(1))
                break
    summary.append({"ligand": ligand_name, "best_score": score})
df = pd.DataFrame(summary).sort_values("best_score")
df.to_csv(OUTPUT_DIR / "docking_summary.csv", index=False)
df.head(20)
